In [1]:
pip install transformers[torch] datasets evaluate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install transformers[torch]

In [3]:
from datasets import load_dataset

arcd = load_dataset("arcd")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/693 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/702 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("arabi-elidrisi/ArabicDistilBERT_QA")

tokenizer_config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/838k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
tokenized_arcd = arcd.map(preprocess_function, batched=True, remove_columns=arcd["train"].column_names)

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

Map:   0%|          | 0/702 [00:00<?, ? examples/s]

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [9]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer


model = AutoModelForQuestionAnswering.from_pretrained("arabi-elidrisi/ArabicDistilBERT_QA")

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [21]:
training_args = TrainingArguments(
    output_dir="mcit",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_arcd["train"],
    eval_dataset=tokenized_arcd["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.225176
2,No log,2.170626
3,No log,2.227480


TrainOutput(global_step=132, training_loss=2.095311482747396, metrics={'train_runtime': 10421.0678, 'train_samples_per_second': 0.199, 'train_steps_per_second': 0.013, 'total_flos': 362036316483072.0, 'train_loss': 2.095311482747396, 'epoch': 3.0})

In [22]:
trainer.save_model("drive/MyDrive/ARABIC-QA")

In [23]:
final_model = AutoModelForQuestionAnswering.from_pretrained("drive/MyDrive/ARABIC-QA")
tokenizer = AutoTokenizer.from_pretrained("drive/MyDrive/ARABIC-QA")

In [14]:
import torch

In [45]:
arcd["train"]["answers"][0]

{'text': ['صحفي وإعلامي'], 'answer_start': [73]}

In [31]:
context = arcd["train"][1]["context"]
question = arcd["train"][1]["question"]

In [35]:
def answer_question(context, question):

  # Preprocess the context and question
  inputs = tokenizer(
    question,
    context,
    max_length=384,
    truncation="only_second",
    return_tensors="pt",
    )

  # Generate the answer using the model
  outputs = final_model(**inputs)
  start_logits = outputs.start_logits
  end_logits = outputs.end_logits


  answer_start = torch.argmax(start_logits, dim=-1).item()
  answer_end = torch.argmax(end_logits, dim=-1).item()

  answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end + 1])
  return answer

# Example usage


predicted_answer = answer_question(context, question)
print(f"Predicted Answer: {predicted_answer}")


Predicted Answer: الحكومة السعودية.


In [52]:
all_context = arcd["train"]["context"]
all_questions = arcd["train"]["question"]
y_pred = []
for context, question in zip(all_context, all_questions):
  y_pred.append(answer_question(context, question))

In [56]:
y_true=[]
for answer in arcd["train"]["answers"]:
  y_true.append(answer["text"][0])

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.16161616161616163

In [65]:
y_true[:10]

['صحفي وإعلامي',
 'حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)،',
 'المدينة المنورة',
 'واشنطن بوست',
 'وُصف في الصحف وأجهزة الاعلام العالمية بأنه "وفيّ للدولة السعودية" و"منتقد لسياساتها".',
 'وتقلّد منصب مستشار،',
 'في سبتمبر 2017،',
 'الحكومة السعودية.',
 'ولي العهد السعودي محمد بن سلمان، والملك سلمان بن عبد العزيز.',
 'المملكة العربية السعودية)']

In [64]:
y_pred[:10]

['صحفي واعلامي سعودي ،',
 '( 13 اكتوبر 1958 ، المدينة المنورة - 2 اكتوبر 2018 ) ،',
 'المدينة المنورة',
 'واشنطن بوست',
 'وصف في الصحف واجهزة الاعلام العالمية بانه " وفي للدولة السعودية " و " منتقد لسياساتها ".',
 'وتقلد منصب مستشار ،',
 'سبتمبر 2017 ،',
 'الحكومة السعودية.',
 'ولي العهد السعودي محمد بن سلمان ، والملك سلمان بن عبد العزيز.',
 'المملكة العربية السعودية )']

In [75]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def compute_semantic_similarity(answer1, answer2):
    embeddings1 = model.encode(answer1, convert_to_tensor=True)
    embeddings2 = model.encode(answer2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embeddings1, embeddings2).item()




Semantic Similarity Score: 1.0


In [86]:
cnt = 0
for true, pred in zip( y_true, y_pred):
  semantic_similarty = compute_semantic_similarity(true, pred)
  if semantic_similarty > 0.8:
    cnt += 1
print("Accuacy",  cnt/len(y_true))

Accuacy 0.8253968253968254


In [80]:
validation = arcd["validation"]

In [81]:
context = validation["context"][0]
print(context)

حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر، وهو خير أعمامه لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».


In [82]:
question = validation["question"][0]
print(question)

من هو حمزة بن عبد المطلب؟


In [87]:
question = "من هو حمزة بن عبد المطلب؟"
context = '''
 حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد،
  وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر،
 وهو خير أعمامه لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».
'''
answer_question(context, question)

'حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الاسلام محمد ،'